In [4]:
import pandas as pd
import numpy as np
from catboost import Pool, cv, CatBoostRegressor, CatBoostClassifier
import autogluon.tabular as agt

In [7]:
predictor = agt.TabularPredictor(label='efs', eval_metric='roc_auc').fit(train.drop(columns='efs_time').iloc[:1000], time_limit=100)

No path specified. Models will be saved in: "AutogluonModels/ag-20241226_172409"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Nov 5 00:21:55 UTC 2024
CPU Count:          16
Memory Avail:       8.54 GB / 15.34 GB (55.7%)
Disk Space Avail:   807.41 GB / 1006.85 GB (80.2%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.

In [11]:
preds = predictor.predict_proba(test, as_pandas=False)
preds

array([[0.88883555, 0.11116447],
       [0.53557754, 0.46442246],
       [0.83915317, 0.16084683]])

In [12]:
preds = [x[1] for x in preds]
preds

[0.11116447299718857, 0.46442246437072754, 0.16084682941436768]

In [5]:
train = pd.read_csv('HCT/train.csv', index_col='ID')
train

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
ID,,,,,,,,,,,,,,,,,,,,,
0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,Intermediate - TED AML case <missing cytogenetics,NaN,Favorable,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,Bi-directional non-permissive,NaN,"N/A, Mel not given",8.0,NaN,2.0,No,10.0,0.0,18.633
28796,High,No,Poor,Yes,1.0,4.0,No TBI,No,5.0,Peripheral blood,...,GvH non-permissive,Related,"N/A, Mel not given",6.0,Yes,1.0,Yes,8.0,1.0,4.892
28797,TBD cytogenetics,NaN,Poor,NaN,2.0,8.0,No TBI,NaN,6.0,Peripheral blood,...,GvH non-permissive,Unrelated,"N/A, Mel not given",8.0,NaN,2.0,No,10.0,0.0,23.157


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28800 entries, 0 to 28799
Data columns (total 59 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dri_score               28646 non-null  object 
 1   psych_disturb           26738 non-null  object 
 2   cyto_score              20732 non-null  object 
 3   diabetes                26681 non-null  object 
 4   hla_match_c_high        24180 non-null  float64
 5   hla_high_res_8          22971 non-null  float64
 6   tbi_status              28800 non-null  object 
 7   arrhythmia              26598 non-null  object 
 8   hla_low_res_6           25530 non-null  float64
 9   graft_type              28800 non-null  object 
 10  vent_hist               28541 non-null  object 
 11  renal_issue             26885 non-null  object 
 12  pulm_severe             26665 non-null  object 
 13  prim_disease_hct        28800 non-null  object 
 14  hla_high_res_6          23516 non-null  flo

In [7]:
train[['efs', 'efs_time']]

,efs,efs_time
ID,,
0,0.0,42.356
1,1.0,4.672
2,0.0,19.793
3,0.0,102.349
4,0.0,16.223
...,...,...
28795,0.0,18.633
28796,1.0,4.892
28797,0.0,23.157


In [8]:
test = pd.read_csv('HCT/test.csv', index_col='ID')
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 28800 to 28802
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dri_score               3 non-null      object 
 1   psych_disturb           3 non-null      object 
 2   cyto_score              1 non-null      object 
 3   diabetes                3 non-null      object 
 4   hla_match_c_high        2 non-null      float64
 5   hla_high_res_8          2 non-null      float64
 6   tbi_status              3 non-null      object 
 7   arrhythmia              3 non-null      object 
 8   hla_low_res_6           3 non-null      float64
 9   graft_type              3 non-null      object 
 10  vent_hist               3 non-null      object 
 11  renal_issue             3 non-null      object 
 12  pulm_severe             3 non-null      object 
 13  prim_disease_hct        3 non-null      object 
 14  hla_high_res_6          3 non-null      flo

In [25]:
train['year_hct'].value_counts()

year_hct
2018    7336
2016    5049
2017    4830
2008    2544
2015    2243
2013    1871
2012    1571
2014    1098
2019     774
2011     599
2009     503
2010     378
2020       4
Name: count, dtype: int64

In [14]:
test

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
ID,,,,,,,,,,,,,,,,,,,,,
28800,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,...,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
28802,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0


In [35]:
pd.read_csv('HCT/data_dictionary.csv')

,variable,description,type,values
0,dri_score,Refined disease risk index,Categorical,['Intermediate' 'High' 'N/A - non-malignant in...
1,psych_disturb,Psychiatric disturbance,Categorical,['Yes' 'No' nan 'Not done']
2,cyto_score,Cytogenetic score,Categorical,['Intermediate' 'Favorable' 'Poor' 'TBD' nan '...
3,diabetes,Diabetes,Categorical,['No' 'Yes' nan 'Not done']
4,hla_match_c_high,Recipient / 1st donor allele level (high resol...,Numerical,NaN
5,hla_high_res_8,Recipient / 1st donor allele-level (high resol...,Numerical,NaN
6,tbi_status,TBI,Categorical,"['No TBI' 'TBI + Cy +- Other' 'TBI +- Other, <..."
7,arrhythmia,Arrhythmia,Categorical,['No' nan 'Yes' 'Not done']
8,hla_low_res_6,Recipient / 1st donor antigen-level (low resol...,Numerical,NaN
9,graft_type,Graft type,Categorical,['Peripheral blood' 'Bone marrow']


In [44]:
cat_cols = train.select_dtypes(include=object).columns.values
cat_cols

array(['prim_disease_hct', 'in_vivo_tcd', 'mrd_hct', 'pulm_severe',
       'cyto_score_detail', 'gvhd_proph', 'dri_score', 'cmv_status',
       'graft_type', 'sex_match', 'conditioning_intensity', 'cyto_score',
       'race_group', 'cardiac', 'prod_type', 'tbi_status'], dtype=object)

In [9]:
num_cols = test.select_dtypes(exclude=object).columns.values

In [5]:
train[cat_cols] = train[cat_cols].fillna('NaN')

In [6]:
test[cat_cols] = test[cat_cols].fillna('NaN')

In [10]:
train[num_cols] = train[num_cols].fillna(-1)
test[num_cols] = test[num_cols].fillna(-1)

In [41]:
X, y = train.drop(columns=['efs', 'efs_time']), train['efs']

KeyError: "['efs', 'efs_time'] not found in axis"

In [42]:
X, y = train, y

In [45]:
train_pool = Pool(X, label=y, cat_features=cat_cols)

In [20]:
model = CatBoostRegressor(task_type='GPU', )

In [21]:
model.fit(train_pool)

Learning rate set to 0.062733
0:	learn: 0.4946363	total: 66.5ms	remaining: 1m 6s
1:	learn: 0.4914058	total: 145ms	remaining: 1m 12s
2:	learn: 0.4884510	total: 249ms	remaining: 1m 22s
3:	learn: 0.4855032	total: 324ms	remaining: 1m 20s
4:	learn: 0.4827042	total: 393ms	remaining: 1m 18s
5:	learn: 0.4802910	total: 479ms	remaining: 1m 19s
6:	learn: 0.4783797	total: 567ms	remaining: 1m 20s
7:	learn: 0.4764766	total: 637ms	remaining: 1m 19s
8:	learn: 0.4749115	total: 723ms	remaining: 1m 19s
9:	learn: 0.4734189	total: 812ms	remaining: 1m 20s
10:	learn: 0.4720281	total: 911ms	remaining: 1m 21s
11:	learn: 0.4706566	total: 997ms	remaining: 1m 22s
12:	learn: 0.4694778	total: 1.08s	remaining: 1m 22s
13:	learn: 0.4684025	total: 1.17s	remaining: 1m 22s
14:	learn: 0.4674249	total: 1.26s	remaining: 1m 22s
15:	learn: 0.4666831	total: 1.34s	remaining: 1m 22s
16:	learn: 0.4658203	total: 1.42s	remaining: 1m 21s
17:	learn: 0.4650135	total: 1.5s	remaining: 1m 21s
18:	learn: 0.4642035	total: 1.56s	remaining: 

In [51]:
model1 = CatBoostClassifier(task_type='GPU', loss_function='Logloss')

In [30]:
model1.fit(train_pool)

Learning rate set to 0.028023
0:	learn: 0.6890781	total: 92.5ms	remaining: 1m 32s
1:	learn: 0.6854417	total: 215ms	remaining: 1m 47s
2:	learn: 0.6819973	total: 335ms	remaining: 1m 51s
3:	learn: 0.6783376	total: 439ms	remaining: 1m 49s
4:	learn: 0.6750115	total: 548ms	remaining: 1m 49s
5:	learn: 0.6718159	total: 642ms	remaining: 1m 46s
6:	learn: 0.6690234	total: 757ms	remaining: 1m 47s
7:	learn: 0.6661348	total: 870ms	remaining: 1m 47s
8:	learn: 0.6634944	total: 981ms	remaining: 1m 48s
9:	learn: 0.6610830	total: 1.06s	remaining: 1m 45s
10:	learn: 0.6586537	total: 1.16s	remaining: 1m 44s
11:	learn: 0.6562918	total: 1.26s	remaining: 1m 44s
12:	learn: 0.6541285	total: 1.37s	remaining: 1m 44s
13:	learn: 0.6521421	total: 1.48s	remaining: 1m 44s
14:	learn: 0.6501615	total: 1.57s	remaining: 1m 43s
15:	learn: 0.6482540	total: 1.68s	remaining: 1m 43s
16:	learn: 0.6465890	total: 1.8s	remaining: 1m 44s
17:	learn: 0.6448728	total: 1.92s	remaining: 1m 44s
18:	learn: 0.6431496	total: 2.02s	remaining:

In [27]:
features0 = pd.DataFrame({'feature': model.feature_names_, 'score': model.feature_importances_}).sort_values(by='score', ascending=False)

In [31]:
features1 = pd.DataFrame({'feature': model1.feature_names_, 'score': model1.feature_importances_}).sort_values(by='score', ascending=False)

In [35]:
features1

,feature,score
26,conditioning_intensity,14.358992
28,year_hct,8.439765
47,karnofsky_score,5.958822
43,sex_match,5.498629
46,comorbidity_score,5.116284
39,age_at_hct,4.695912
15,cmv_status,3.721149
13,prim_disease_hct,3.478282
41,gvhd_proph,3.291032
0,dri_score,3.259425


In [39]:
features = list(set(features0['feature'].iloc[:25]) & set(features1['feature'].iloc[:25]))

In [40]:
train = train[features]
test = test[features]

In [54]:
features

['prim_disease_hct',
 'donor_age',
 'in_vivo_tcd',
 'hla_match_drb1_high',
 'mrd_hct',
 'pulm_severe',
 'hla_match_a_high',
 'cyto_score_detail',
 'gvhd_proph',
 'comorbidity_score',
 'dri_score',
 'cmv_status',
 'graft_type',
 'sex_match',
 'conditioning_intensity',
 'cyto_score',
 'year_hct',
 'race_group',
 'hla_nmdp_6',
 'cardiac',
 'prod_type',
 'tbi_status',
 'karnofsky_score',
 'hla_match_dqb1_high',
 'age_at_hct']

In [52]:
cv_data = cv(train_pool, model1.get_params(),)

Training on fold [0/3]
0:	learn: 0.6889450	test: 0.6889339	best: 0.6889339 (0)	total: 58.2ms	remaining: 58.2s
1:	learn: 0.6850364	test: 0.6850854	best: 0.6850854 (1)	total: 118ms	remaining: 58.8s
2:	learn: 0.6815281	test: 0.6816086	best: 0.6816086 (2)	total: 205ms	remaining: 1m 8s
3:	learn: 0.6778416	test: 0.6778712	best: 0.6778712 (3)	total: 286ms	remaining: 1m 11s
4:	learn: 0.6747223	test: 0.6747814	best: 0.6747814 (4)	total: 364ms	remaining: 1m 12s
5:	learn: 0.6714229	test: 0.6714334	best: 0.6714334 (5)	total: 431ms	remaining: 1m 11s
6:	learn: 0.6685042	test: 0.6686460	best: 0.6686460 (6)	total: 511ms	remaining: 1m 12s
7:	learn: 0.6656146	test: 0.6658547	best: 0.6658547 (7)	total: 591ms	remaining: 1m 13s
8:	learn: 0.6627824	test: 0.6629846	best: 0.6629846 (8)	total: 660ms	remaining: 1m 12s
9:	learn: 0.6603316	test: 0.6606701	best: 0.6606701 (9)	total: 743ms	remaining: 1m 13s
10:	learn: 0.6579191	test: 0.6583321	best: 0.6583321 (10)	total: 808ms	remaining: 1m 12s
11:	learn: 0.6558523

In [55]:
cv_data['test-Logloss-mean']

0      0.689015
1      0.685287
2      0.681782
3      0.678134
4      0.674940
         ...   
995    0.584359
996    0.584357
997    0.584356
998    0.584352
999    0.584352
Name: test-Logloss-mean, Length: 1000, dtype: float64

In [ ]:
import optuna


def objective(trial: optuna.Trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 0.1, log=True),
        'depth': trial.suggest_int('depth', 3, 11),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 0.1, log=True),
        'random_state': trial.suggest_categorical('random_state', range(100)),
        'task_type': 'GPU'
             }
    model = CatBoostClassifier(**params, loss_function='Logloss', silent=True)
    cv_data = cv(train_pool, model.get_params())
    return cv_data['test-Logloss-mean'].values[-1]

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2024-12-26 18:37:25,131] A new study created in memory with name: no-name-5c9f1008-b9fc-4859-b583-4c34988be4b7


Training on fold [0/3]
0:	learn: 0.6900032	test: 0.6901784	best: 0.6901784 (0)	total: 222ms	remaining: 3m 41s
1:	learn: 0.6865591	test: 0.6870888	best: 0.6870888 (1)	total: 427ms	remaining: 3m 33s
2:	learn: 0.6836916	test: 0.6842877	best: 0.6842877 (2)	total: 513ms	remaining: 2m 50s
3:	learn: 0.6809194	test: 0.6815924	best: 0.6815924 (3)	total: 727ms	remaining: 3m 1s
4:	learn: 0.6782471	test: 0.6789170	best: 0.6789170 (4)	total: 818ms	remaining: 2m 42s
5:	learn: 0.6753583	test: 0.6763521	best: 0.6763521 (5)	total: 1.03s	remaining: 2m 50s
6:	learn: 0.6727305	test: 0.6736276	best: 0.6736276 (6)	total: 1.24s	remaining: 2m 56s
7:	learn: 0.6703901	test: 0.6713346	best: 0.6713346 (7)	total: 1.46s	remaining: 3m
8:	learn: 0.6681573	test: 0.6691963	best: 0.6691963 (8)	total: 1.66s	remaining: 3m 2s
9:	learn: 0.6658322	test: 0.6669192	best: 0.6669192 (9)	total: 1.87s	remaining: 3m 4s
10:	learn: 0.6637634	test: 0.6648294	best: 0.6648294 (10)	total: 1.94s	remaining: 2m 54s
11:	learn: 0.6617680	test

KeyboardInterrupt: 

Exception ignored in: '_catboost._WriteLog'
Traceback (most recent call last):
  File "/home/xbar19/ML/MLEnv/lib/python3.10/site-packages/ipykernel/iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


KeyboardInterrupt: 

...

44:	learn: 0.6194620	test: 0.6253688	best: 0.6253688 (44)	total: 7.99s	remaining: 2m 49s
45:	learn: 0.6186793	test: 0.6247348	best: 0.6247348 (45)	total: 8.21s	remaining: 2m 50s
46:	learn: 0.6182703	test: 0.6243752	best: 0.6243752 (46)	total: 8.27s	remaining: 2m 47s
47:	learn: 0.6176830	test: 0.6237968	best: 0.6237968 (47)	total: 8.47s	remaining: 2m 48s
48:	learn: 0.6169690	test: 0.6232536	best: 0.6232536 (48)	total: 8.7s	remaining: 2m 48s
49:	learn: 0.6162049	test: 0.6226745	best: 0.6226745 (49)	total: 8.91s	remaining: 2m 49s
50:	learn: 0.6155280	test: 0.6220286	best: 0.6220286 (50)	total: 9.12s	remaining: 2m 49s
51:	learn: 0.6146689	test: 0.6214248	best: 0.6214248 (51)	total: 9.33s	remaining: 2m 50s
52:	learn: 0.6140110	test: 0.6209063	best: 0.6209063 (52)	total: 9.54s	remaining: 2m 50s
53:	learn: 0.6133301	test: 0.6204286	best: 0.6204286 (53)	total: 9.76s	remaining: 2m 50s
54:	learn: 0.6126537	test: 0.6199417	best: 0.6199417 (54)	total: 9.96s	remaining: 2m 51s
55:	learn: 0.6121082	t

[I 2024-12-26 18:47:19,039] Trial 0 finished with value: 0.5856587741559 and parameters: {'learning_rate': 0.02134062386206166, 'depth': 9, 'l2_leaf_reg': 0.021816786767475213, 'random_state': 82}. Best is trial 0 with value: 0.5856587741559.


999:	learn: 0.5089410	test: 0.5823978	best: 0.5823814 (939)	total: 3m 20s	remaining: 0us
bestTest = 0.582381374
bestIteration = 939
Training on fold [0/3]
0:	learn: 0.6903140	test: 0.6902850	best: 0.6902850 (0)	total: 61ms	remaining: 1m
1:	learn: 0.6874961	test: 0.6874987	best: 0.6874987 (1)	total: 127ms	remaining: 1m 3s
2:	learn: 0.6847981	test: 0.6848158	best: 0.6848158 (2)	total: 183ms	remaining: 1m
3:	learn: 0.6824678	test: 0.6825489	best: 0.6825489 (3)	total: 256ms	remaining: 1m 3s
4:	learn: 0.6799564	test: 0.6799834	best: 0.6799834 (4)	total: 340ms	remaining: 1m 7s
5:	learn: 0.6779517	test: 0.6779680	best: 0.6779680 (5)	total: 380ms	remaining: 1m 2s
6:	learn: 0.6757955	test: 0.6758336	best: 0.6758336 (6)	total: 454ms	remaining: 1m 4s
7:	learn: 0.6735890	test: 0.6736416	best: 0.6736416 (7)	total: 535ms	remaining: 1m 6s
8:	learn: 0.6713872	test: 0.6715100	best: 0.6715100 (8)	total: 612ms	remaining: 1m 7s
9:	learn: 0.6693033	test: 0.6694628	best: 0.6694628 (9)	total: 679ms	remaining

[I 2024-12-26 18:51:07,820] Trial 1 finished with value: 0.5866062933640787 and parameters: {'learning_rate': 0.02027718660710031, 'depth': 6, 'l2_leaf_reg': 0.0017106662962572206, 'random_state': 21}. Best is trial 0 with value: 0.5856587741559.


Training on fold [0/3]
0:	learn: 0.6928095	test: 0.6928825	best: 0.6928825 (0)	total: 675ms	remaining: 11m 14s
1:	learn: 0.6925364	test: 0.6926279	best: 0.6926279 (1)	total: 925ms	remaining: 7m 41s
2:	learn: 0.6922883	test: 0.6923922	best: 0.6923922 (2)	total: 1.32s	remaining: 7m 19s
3:	learn: 0.6921021	test: 0.6922020	best: 0.6922020 (3)	total: 1.39s	remaining: 5m 45s
4:	learn: 0.6917995	test: 0.6919341	best: 0.6919341 (4)	total: 1.82s	remaining: 6m 1s
5:	learn: 0.6915394	test: 0.6916950	best: 0.6916950 (5)	total: 1.99s	remaining: 5m 30s
6:	learn: 0.6913544	test: 0.6915076	best: 0.6915076 (6)	total: 2.06s	remaining: 4m 51s
7:	learn: 0.6911534	test: 0.6913075	best: 0.6913075 (7)	total: 2.13s	remaining: 4m 23s
8:	learn: 0.6909285	test: 0.6910812	best: 0.6910812 (8)	total: 2.23s	remaining: 4m 5s
9:	learn: 0.6907290	test: 0.6908810	best: 0.6908810 (9)	total: 2.29s	remaining: 3m 47s
10:	learn: 0.6904907	test: 0.6906485	best: 0.6906485 (10)	total: 2.46s	remaining: 3m 41s
11:	learn: 0.690201

In [15]:
test_pool = Pool(test, cat_features=cat_cols)

In [16]:
preds = model.predict(test_pool)
preds

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

In [32]:
sub = pd.read_csv('HCT/sample_submission.csv', index_col='ID')
sub

,prediction
ID,
28800,0.5
28801,0.5
28802,0.5


In [33]:
sub['prediction'] = preds

In [34]:
sub.to_csv('sub_health0.csv')